In [1]:
import langchain

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
os.getenv("GROQ_API_KEY")

### Example 1: Simple LLM calls with streaming

In [4]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage,SystemMessage

In [7]:
model = init_chat_model("groq:llama-3.1-8b-instant")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002D67891C190>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002D67891CB90>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [8]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="llama-3.1-8b-instant")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002D675F37ED0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002D67891D810>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [12]:
## create messages
messages=[
    SystemMessage("You are a helpful AI assistant"),
    HumanMessage("What are the 2 benefits of using Langchain?")
]

In [13]:
## invoke the model
response = model.invoke(messages)
response

AIMessage(content='LangChain is an open-source framework that enables the building of large language models and conversational AI applications. \n\nTwo benefits of using LangChain are:\n\n1. **Building Complex Conversational AI Applications**: LangChain offers a flexible and modular architecture that allows developers to create sophisticated conversational AI models that can handle multiple tasks, such as answering questions, generating text, and performing tasks. This enables the creation of more realistic and interactive chatbots and virtual assistants.\n\n2. **Scalability and Customizability**: LangChain provides a framework for integrating and combining multiple large language models and APIs, allowing developers to customize and fine-tune their conversational AI applications to meet specific requirements. This enables the development of more accurate and efficient conversational AI models that can adapt to different use cases and domains.', additional_kwargs={}, response_metadata=

In [14]:
print(response.content)

LangChain is an open-source framework that enables the building of large language models and conversational AI applications. 

Two benefits of using LangChain are:

1. **Building Complex Conversational AI Applications**: LangChain offers a flexible and modular architecture that allows developers to create sophisticated conversational AI models that can handle multiple tasks, such as answering questions, generating text, and performing tasks. This enables the creation of more realistic and interactive chatbots and virtual assistants.

2. **Scalability and Customizability**: LangChain provides a framework for integrating and combining multiple large language models and APIs, allowing developers to customize and fine-tune their conversational AI applications to meet specific requirements. This enables the development of more accurate and efficient conversational AI models that can adapt to different use cases and domains.


In [18]:
## streaming example

for chunk in model.stream(messages):
    print(chunk.content,end="",flush=True)

Langchain is an AI platform that enables the creation of advanced conversational AI models. Two benefits of using Langchain include:

1. **Improved Conversational Capabilities**: Langchain's platform allows developers to build more sophisticated conversational AI models by combining multiple AI models and technologies, such as natural language processing (NLP), large language models (LLMs), and knowledge graphs. This enables more natural and effective human-computer interactions.

2. **Enhanced Knowledge Integration**: Langchain's ability to integrate multiple AI models and knowledge sources allows for more comprehensive and up-to-date information to be used in conversational AI models. This results in more informed and accurate responses to user queries, making it a valuable tool for applications such as customer service, technical support, and more.

### Dynamic prompt templates

In [21]:
from langchain_core.prompts import ChatPromptTemplate

## create a translation app

translation_template = ChatPromptTemplate.from_messages([
    ("system","You are a professional translator. Translate the following text {text} from {source_language} to {target_language}.Maintain the tone and style"),
    ("user","{text}")
])

## using the template

prompt = translation_template.invoke({
    "source_language":"English",
    "target_language":"Spanish",
    "text":"Langchain makes building AI Application incredibly easy!"
})

In [22]:
prompt

ChatPromptValue(messages=[SystemMessage(content='You are a professional translator. Translate the following text Langchain makes building AI Application incredibly easy! from English to Spanish.Maintain the tone and style', additional_kwargs={}, response_metadata={}), HumanMessage(content='Langchain makes building AI Application incredibly easy!', additional_kwargs={}, response_metadata={})])

In [25]:
translated_resp = model.invoke(prompt)
translated_resp.content

'Langchain hace que construir aplicaciones de IA sea increíblemente fácil.'

### Building chains

In [46]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough,RunnableLambda

def create_story_chain():
    story_prompt = ChatPromptTemplate.from_messages([
        ("system","You are a creative storyteller.Write a short and engaging story based on the given theme"),
        ("user","Theme: {theme}\n Main character: {character}\n settings: {settings}")
    ])
    analysis_prompt = ChatPromptTemplate.from_messages([
        ("system","You are a literacy critic. Analyze the following story and provide insights"),
        ("user","{story}")
    ])

    story_chain = (
        story_prompt | model | StrOutputParser()
    )

    def analyze_text(story_text):
        return {"story":story_text}

    analysis_chain = (
        story_chain
        | RunnableLambda(analyze_text)
        | analysis_prompt
        | model
        | StrOutputParser()
    )

    return analysis_chain

In [47]:
chain =create_story_chain()
chain

ChatPromptTemplate(input_variables=['character', 'settings', 'theme'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a creative storyteller.Write a short and engaging story based on the given theme'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['character', 'settings', 'theme'], input_types={}, partial_variables={}, template='Theme: {theme}\n Main character: {character}\n settings: {settings}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002D67891C190>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002D67891CB90>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()
| RunnableLambda(analyze_text)
| ChatPromptTemplate(input_variables=['story'], input_types={}, partial_var

In [49]:
result = chain.invoke({
    "theme":"Artificial intelligence",
    "character":"A curious robot",
    "settings":"A futuristic city"
})

print("---------------Story and Analysis--------------")
print(result)

---------------Story and Analysis--------------
**Analysis of "The Curious Robot of NeuroCity"**

"The Curious Robot of NeuroCity" is a thought-provoking science fiction story that explores the themes of artificial intelligence, human innovation, and the ethics of technological advancements. On the surface, the narrative appears to be a straightforward tale of a curious robot named Zeta who uncovers the secrets of the NeuroCore, a revolutionary brain-computer interface. However, upon closer examination, the story reveals itself to be a nuanced exploration of the complexities of human-technology interactions.

**Themes and Symbolism**

1. **The Double-Edged Sword of Technology**: The NeuroCore represents a double-edged sword, capable of enhancing human intelligence or creating a new form of dependence on technology. This theme is reminiscent of Mary Shelley's Frankenstein, where the protagonist's creation brings about both wonder and terror.
2. **The Quest for Knowledge**: Zeta's curios